Stacked Learning 1st Part

In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.metrics import accuracy_score

#1. Load DataSet
iris = load_iris()
X, y = iris.data, iris.target

#2. Split DataSet
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#3. Define base learners
base_learners = [
    ('dt', DecisionTreeClassifier()),
    ('svc', SVC(kernel='rbf', probability=True)),
    ('rf', RandomForestClassifier(n_estimators=100))
]

#4. Define meta-learner (final estimator)
meta_learner = LogisticRegression()

#5. Create StackingClassifier
stacked_model = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_learner,
    cv=5,     #cross-validation for base models
    passthrough=False, #if True, passes original features along with base outputs
    n_jobs=-1
)

#6. Train Stacked Model
stacked_model.fit(X_train, y_train)

#7. Make Predictions
y_pred = stacked_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Stacked Model Accuracy:", acc)



Stacked Model Accuracy: 1.0


Stacked Learning 2nd Part

In [7]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

#1. Load Dataset
iris = load_iris()
X, y = iris.data, iris.target
y_cat = to_categorical(y, num_classes=3)

#2. Split into Train and Test
X_train, X_test, y_train_cat, y_test_cat, y_train, y_test = train_test_split(
    X, y_cat, y, test_size=0.2, random_state=4
)

#3. Define Function to create a model
def create_model_1():
  model = Sequential([
      Dense(16, input_shape=(4,), activation='relu'),
      Dense(3, activation='softmax')
  ])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model
def create_model_2():
  model = Sequential([
      Dense(32, input_shape=(4,), activation='relu'),
      Dense(16, activation='relu'),
      Dense(3, activation='softmax')
  ])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

#4. Train Base Models
model1 = create_model_1()
model2 = create_model_2()

model1.fit(X_train, y_train_cat, epochs=50, verbose=0)
model2.fit(X_train, y_train_cat, epochs=50, verbose=0)

#5. Get Predictions for training data (to train meta-model)
train_pred1 = model1.predict(X_train)
train_pred2 = model2.predict(X_train)

#Stack predictions horizontally to create meta-features
stacked_train_features = np.hstack((train_pred1, train_pred2))

#6. Train meta-model on predictions
meta_model = LogisticRegression(max_iter=1000)
meta_model.fit(stacked_train_features, y_train)

#7. Predict on test data
test_pred1 = model1.predict(X_test)
test_pred2 = model2.predict(X_test)

stacked_test_features = np.hstack((test_pred1, test_pred2))
final_pred = meta_model.predict(stacked_test_features)

#8. Accuracy
print("Stacked Deep Learning Esemble Accuracy:", accuracy_score(y_test, final_pred))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Stacked Deep Learning Esemble Accuracy: 0.9333333333333333
